<img src=../images/gdms-vault.png width=1000 margin=0px>

---



# Performance Replication in Vault

**Cleanup Directories**

In [9]:
export DEMODIR=~/work/jupyter/hc_demos-jupyter
export VLICENSE=~/.licenses/vault.json

In [12]:
rm -rf $DEMODIR/vault/config/vrd/*
rm -rf $DEMODIR/vault/config/vrd2/*

**Setup Directories**

In [11]:
mkdir -p $DEMODIR/vault/config
mkdir -p $DEMODIR/vault/config/vrd/data/vault/raft
mkdir -p $DEMODIR/vault/config/vrd2/data/vault/raft
mkdir -p $DEMODIR/vault/config/vrd/data/vault/plugins
mkdir -p $DEMODIR/vault/config/vrd2/data/vault/plugins

### Begin Performance Replication

In [4]:
vault version

Vault v1.5.0+ent


In [5]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [6]:
export VAULT_ADDR2=http://127.0.0.1:8202
export VAULT_SKIP_VERIFY=true

### Start Two Vault Servers

**Populate Vault Server configuration files**

In [7]:
mkdir -p $DEMODIR/vault/config
cat <<EOF > $DEMODIR/vault/config/vrd_config.hcl
storage "raft" {
path = "$DEMODIR/vault/config/vrd/data/vault/raft"
  node_id = "demo"
}
listener "tcp" {
  address = "0.0.0.0:8200"
  cluster_address = "0.0.0.0:8201"
  tls_disable = "true"
}
api_addr = "https://127.0.0.1:8200"
cluster_addr = "https://127.0.0.1:8201"
disable_mlock="false"
disable_cache="false"
ui = "true"
max_lease_ttl="24h"
default_lease_ttl="1h"
raw_storage_endpoint=true
cluster_name="hashi-vault"
insecure_tls="true"
plugin_directory="$DEMODIR/vault/config/vrd/data/vault/plugins"
EOF

In [8]:
mkdir -p $DEMODIR/vault/config
cat <<EOF > $DEMODIR/vault/config/vrd2_config.hcl
storage "raft" {
  path = "$DEMODIR/vault/config/vrd2/data/vault/raft"
  node_id = "demo"
}
listener "tcp" {
  address = "0.0.0.0:8202"
  cluster_address = "0.0.0.0:8203"
  tls_disable = "true"
}
api_addr = "https://127.0.0.1:8202"
cluster_addr = "https://127.0.0.1:8203"
disable_mlock="false"
disable_cache="false"
ui = "true"
max_lease_ttl="24h"
default_lease_ttl="1h"
raw_storage_endpoint=true
cluster_name="hashi-vault"
insecure_tls="true"
plugin_directory="$DEMODIR/vault/config/vr2/data/vault/plugins"
EOF

**AUTO RUN ABOVE**

**EXECUTE EACH COMMAND IN SEPARATE TERMINAL WINDOW**

In [ ]:
vault server -config=$DEMODIR/vault/config/vrd_config.hcl

In [ ]:
vault server -config=$DEMODIR/vault/config/vrd2_config.hcl

### Initialize Vaults
(save output to file for demo use)

In [ ]:
curl --request POST --data '{"secret_shares": 1, "secret_threshold": 1}' $VAULT_ADDR/v1/sys/init > $DEMODIR/vault/config/vrd/vault_init_output

In [ ]:
curl --request POST --data '{"secret_shares": 1, "secret_threshold": 1}' $VAULT_ADDR2/v1/sys/init > $DEMODIR/vault/config/vrd2/vault_init_output

**Parse output to variables** (for demo use)

In [ ]:
sleep 10
export VAULT_ROOT_TOKEN=$(cat $DEMODIR/vault/config/vrd/vault_init_output | jq -r '.root_token')
export VAULT_KEY_BASE64=$(cat $DEMODIR/vault/config/vrd/vault_init_output | jq -r '.keys_base64[0]' )

In [ ]:
export VAULT_ROOT_TOKEN2=$(cat $DEMODIR/vault/config/vrd2/vault_init_output | jq -r '.root_token')
export VAULT_KEY_BASE642=$(cat $DEMODIR/vault/config/vrd2/vault_init_output | jq -r '.keys_base64[0]' )

### Unseal Vaults

In [ ]:
  sleep 10;
  curl \
    --insecure \
    --header "X-Vault-Token: $VAULT_ROOT_TOKEN" \
    --request PUT \
    --data '{ "key": "'$VAULT_KEY_BASE64'" }' \
    $VAULT_ADDR/v1/sys/unseal

In [ ]:
  curl \
    --insecure \
    --header "X-Vault-Token: $VAULT_ROOT_TOKEN2" \
    --request PUT \
    --data '{ "key": "'$VAULT_KEY_BASE642'" }' \
    $VAULT_ADDR2/v1/sys/unseal

### License Vaults

In [ ]:
  sleep 10
  curl \
    --insecure \
    --header "X-Vault-Token: $VAULT_ROOT_TOKEN" \
    --request PUT \
    --data "@$VLICENSE" \
    $VAULT_ADDR/v1/sys/license

In [ ]:
  curl \
    --insecure \
    --header "X-Vault-Token: $VAULT_ROOT_TOKEN2" \
    --request PUT \
    --data "@$VLICENSE" \
    $VAULT_ADDR2/v1/sys/license

# Interactive Demo: Enable Performance Replication

**Copy Token to Pasteboard for UI Login**

In [ ]:
echo $VAULT_ROOT_TOKEN | pbcopy
echo $VAULT_ROOT_TOKEN

In [ ]:
echo $VAULT_ROOT_TOKEN2 | pbcopy

**Generate NEW root key for Performance Replication cluster**

In [ ]:
## Generate One Time Password (OTP) Needed to Generate DR token
#alias vault_secondary='VAULT_UI=true VAULT_ADDR=http://127.0.0.1:8202 vault'
#VAULT_SECONDARY_ADDR='127.0.0.1:8202'
#ONE_TIME_PASSWORD=$(vault_secondary operator generate-root -generate-otp)
ONE_TIME_PASSWORD=$(vault operator generate-root -generate-otp VAULT_ADDR=http://127.0.0.1:8202)
echo $ONE_TIME_PASSWORD

In [ ]:
## Start Generation of DR Operation Token Attempt -- Get NONCE to give to UNSEAL KEY holders
NONCE=$(curl --header "X-Vault-Token: $VAULT_ROOT_TOKEN" --request PUT --data '{"otp":"'"${ONE_TIME_PASSWORD}"'"}' 127.0.0.1:8202/v1/sys/generate-root/attempt | jq --raw-output '.nonce')
echo $NONCE

In [ ]:
## Get Your ENCODED TOKEN that will be combined with OTP to produce New Root Token for Perf Replica cluster
export UNSEAL_KEY=$(cat ~/Documents/demos/vault_cluster/vrd/vault_init_output | jq -r '.keys[0]')
echo $UNSEAL_KEY
ENCODED_TOKEN=$(curl --header "X-Vault-Token: ${VAULT_ROOT_TOKEN}" --request PUT --data '{"key":"'"${UNSEAL_KEY}"'", "nonce":"'"${NONCE}"'"}' http://127.0.0.1:8202/v1/sys/generate-root/update | jq --raw-output '.encoded_token')
echo $ENCODED_TOKEN
#PR_ROOT_TOKEN=$(vault_secondary operator generate-root -otp="${ONE_TIME_PASSWORD}" -decode="${ENCODED_TOKEN}")
PR_ROOT_TOKEN=$(VAULT_ADDR=http://127.0.0.1:8202 vault operator generate-root -otp="${ONE_TIME_PASSWORD}" -decode="${ENCODED_TOKEN}")
echo $PR_ROOT_TOKEN

In [ ]:
echo $PR_ROOT_TOKEN | pbcopy

## Interactive Demo: Log Into Replica
## Interactive Demo: Create / Delete Secrets in Primary and View in Replica

**END**